## Chapter2 The Graphics Rendering Pipeline

### Architecture
- **The pipeline stages execute in parallel, with each stage dependent upon the result of the previous stage.**
- 例如,一批人可以加工大量的三明治--- 一个准备面包,另一个加肉,另一个加配料.每个人将自己加工完成的三明治传递给流水线上的下一个人并直接开始加工下一个三明治.如果每个人执行自己的任务要花费20秒,那么最高效率下,20秒就可以产出一个三明治,一分钟生产三个也是可能的.虽然管线各阶段并行执行,但它们会被其中最慢的一个阶段阻塞.例如,加工肉的阶段变得比较复杂,要花费30秒,那么现在最高的产出率就是一分钟两个三明治.对于这种特殊的生成流水线来说,加工肉的阶段就成为了瓶颈.
- 实时渲染管线可以粗略分为四个阶段--- 应用层,几何体处理,光栅化和像素处理![Rendering Pipeline](./Images/RTR.2.2.png)  
- 光栅化阶段通常将三个顶点作为输入,把它们格式化为一个三角形,并找出所有包含在三角形内的像素点,然后传输给下一个阶段.
- 最终,像素处理阶段为每个像素执行一段程序来决定像素的颜色并有可能执行深度测试来决定这个像素是否可见.这个阶段也有可能执行一些逐像素操作,例如将新计算出的颜色与之前的颜色混合.

### The Application Stage
- 应用层阶段通常在cpu上执行,在应用层阶段的末尾,需要渲染的集合体被传输给几何处理阶段,这些是渲染图元,也就是点,线和三角形.

### Geometry Processing
- 运行于GPU上的几何处理阶段负责大部分的逐三角形和逐顶点操作.这个阶段可细分为以下几个部分:顶点着色,投影,裁剪和屏幕映射.![Geometry Processing](./Images/RTR.2.3.png)  

#### Vertex Shading
- 顶点着色有两个主要任务:计算顶点的位置和输出数据,例如法线和纹理坐标.典型的为物体着色就是将光照信息应用到顶点的位置和法线,并只在顶点中存储最终颜色.然后在三角形中根据顶点颜色进行插值.随着现代GPU的出现,部分或全部这种类型的着色被逐像素着色所取代,这个顶点着色阶段或许完全不再执行着色方程,这取决于程序员的意愿.目前的Vertex Shader是一个用于配置每个顶点相关数据的更通用的单元.例如处理物体动画.
- 在显示到屏幕上的过程中,一个模型被变换到几个不同的空间和坐标系统中.开始时,模型位于模型空间中.模型变换作用于模型的顶点和法线.一个物体的坐标系叫做模型坐标系,当模型变换应用到这些模型坐标系之后,就称模型被置于到了世界坐标系或者世界空间中.世界空间是唯一的,并且在所有的模型经过它们各自的模型变换之后都会处于同一个空间中.
- 只有摄像机或者观察者能看到的模型才会被渲染.摄像机被放置于世界空间中, 为了投影和裁剪,摄像机和所有的模型都被应用了观察变换(view transform).观察变换的目的是把摄像机放置到原点并指向z轴的反方向,同时y轴指向上方,x轴指向右方.我们以z轴指向屏幕内为惯例,有些描述倾向于使z轴指向屏幕外.这个空间叫做摄像机空间或者更通用的叫法是观察空间或视觉空间.
- 渲染系统还要执行投影和裁剪,将观察体变换到$(-1,-1,-1)$到$(1,1,1)$的范围内.可以用不同的表示范围定义同一个观察体,例如($0<=z<=1$).
- 单位立方体叫做规范观察体(cvv, canonical view volume).
- 投影在GPU端被Vertex Shader首先完成.通常有两种投影方式,分别叫做正交(或平行)和透视投影.实际上,正交投影只是平行投影的一种.其他几种平行投影方式多在建筑学领域应用,例如斜投影和轴测投影.    
- 正交投影的观察空间一般是一个长方体盒子,正交投影变换将观察体映射到单位立方体中.正交投影的主要特征是平行线在变换之后依然保持平行.这种变换是平移变换和缩放变换的结合.  
- 透视投影更复杂一些.离摄像机越远的物体越小,平行线经变换后可能会在地平线处相交.因此.透视投影用来模拟我们观察物体的方式.其观察体叫做平截头体(frustum),平截头体也会变换到单位立方体.
- 在两者中的任一变换后,我们就说模型被放置在了裁剪坐标系中.这些实际是齐次坐标系.所有这些发生在齐次除法之前.GPU的Vertex Shader必须总是输出这种类型的坐标系,为了保证正确的裁剪.

#### Optional Vertex Processing
- 还有一些可选的阶段可以在GPU端以以下顺序执行:曲面细分,几何着色和流输出.它们是否能被使用都依赖于硬件的能力,不是所有的gpu都有这些功能.
- Geometry Shader跟曲面细分的Shader有些相似,它以不同类型的图元作为输入来产生新的顶点.但它限定于特定的阶段并且对于输出的图元类型也有限制.Geometry Shader有多种用途,其中最常用的一种是生成粒子.设想模拟烟火爆炸,每一个火球都可以用一个顶点表示.geometry shader可以把这些顶点转换成朝向观察者且覆盖了一些像素点的方形(两个三角形组成),以提供一个更合适的图元来着色.

#### Clipping
- 应用投影矩阵意味着转换过的图元被单位立方体裁剪.在裁剪之前应用观察变换和投影变换的好处是:图元总是按照单位立方体来裁剪.除了观察体的六个裁剪平面之外,用户也可以定义额外的裁剪平面.![Clipping](./Images/RTR.2.6.png)
- 裁剪使用由投影生成的齐次坐标来执行.
- 最后执行透视除法,将三角形的位置信息映射到三维的归一化了的设备坐标系中.
- 几何阶段的最后一步是转换到窗口坐标系.

#### Screen Mapping
- 只有观察体中的图元被传输到屏幕映射阶段,并且进入这个阶段时坐标系仍然是三维的.每个图元的x和y坐标轴都被转换到屏幕坐标系.屏幕坐标系加上z轴也叫窗体坐标系.
- 假设场景被绘制到一个最小角为$(x_{1},y_{1})$,最大角为$(x_{2},y_{2})$的窗体上,那么屏幕映射就是一个缩放变换跟着一个平移变换.新的x轴和y轴组成屏幕坐标系.对于OpenGL来说$[-1,1]$也被映射到$[z_{1},z{2}]$(对DirectX来说是$[0,1]$),默认情况下$z_{1}=0$,$z_{2}=1$.窗体坐标系和被重新映射过的z值传输到光栅化阶段.屏幕映射的过程如图2.7所示.![Screen Mapping](./Images/RTR.2.7.png)
- 给定一个笛卡尔坐标系下的水平像素数组,最左边的像素是0.0.像素的中心是0.5.于是$[0,9]$范围内的像素覆盖了$[0.0,10.0]$的区域.转换方程如下:
$$d=floor(c)$$ $$c=d+0.5$$ d是像素一个离散的索引,c是像素内连续的值.  
- 对OpenGL和DirectX来说顶部和底部的零值在某些情况下不一致.OpenGL倾向于使用笛卡尔坐标系,以左下角为最小位置,但DirectX有时以左上角为最小位置,要依据上下文来决定.

### Rasterization
- 给定经过变换和投影的顶点以及它们的着色数据(所有这些数据都来自于几何处理阶段),下一个阶段的目标是找出图元内的所有元素,这一阶段叫做光栅化,并且可分为两个子阶段:三角形设置(也叫做图元装备)和三角形遍历.![Rasterization](./Images/RTR.2.8.png)
- 也可以处理点和线,但通常是三角形.光栅化也叫做扫描转换,即从屏幕空间的带有z值的二维顶点以及和顶点相关的多种着色信息转换成屏幕上的像素.光栅化也可以被当做几何处理阶段和像素处理阶段的同步点.  
- 三角形是否重叠取决于你怎样设置管线.例如,你可能用采样一个点来决定这个点是否在图元内.最简单的情况是在每个像素中心采样一个点,这样如果这个中心点在三角形内,那么其对应的像素点也在三角形内.你也可能用超采样或多采样抗锯齿技术采样像素中的多个点.另一种方式是采用保守的光栅化,即至少像素的一部分在与三角形重叠才能说它在三角形内.

#### Triangle Setup
- 这个阶段计算边方程和和三角形的其他数据.这些数据可能用于三角形遍历,也可用于对几何阶段产生的多种着色数据进行插值.固定功能的硬件负责这个任务.

#### Triangle Traversal
- 找出哪些采样点或像素点在三角形内的过程通常叫做三角形遍历.每个三角形片元的属性都是通过对三个顶点数据插值得出的.这些属性包括片元的深度,也包括任何从几何阶段得出的着色数据.所有在图元内的像素点或采样点被传输到像素处理阶段.

### Pixel Processing
#### Pixel Shading
- 这个阶段负责所有逐像素的着色计算,使用插值过的着色数据作为输入,输出一个或多个颜色到下一个阶段.与三角形设置和三角形遍历阶段不同的是通常使用特质的芯片来执行,像素着色阶段由可编程GPU核心来执行.

#### Merging
- 每个像素的信息被存在颜色缓存中.合并阶段的功能是将像素着色阶段产生的片元颜色和缓存中当前存储的颜色进行合并.这个阶段也叫做ROP(raster operation or render output unit).这个阶段通常不是完全可编程的,但是高度可配置的,允许多种效果.
- 这个阶段也负责判定可见性.这就是说颜色缓冲中必须包含场景中从摄像机视角来看的所有可见图元的颜色.对大多数甚至所有的图形硬件来说,这是由Z-Buffer算法完成的.Z-Buffer拥有Color-Buffer同样的尺寸和形状,并且对每个像素存储一个当前最近的图元的Z值.这就是说当一个图元被绘制到一个指定的像素时,图元在这个像素点的Z值将会被计算并拿来与Z-Buffer中的同一个位置的像素点的Z值做比较,如果新的Z值比Z-Buffer中的Z值小,那么将要渲染的图元比前一个距离摄像机最近的像素点还要近.因此,这个像素点的Z值和颜色将被更新.反之,则会被保留.Z-Buffer的算法很简单,这个算法允许大多数图元无须渲染,这也是它流行的另一个原因.
- Z-Buffer仅为屏幕上每个点存储一个深度值,所以它不能用于半透明的图元.必须在所有不透明的图元绘制完成后,并用从后往前的顺序绘制半透明图元,或者使用顺序无关的算法.半透明物体的绘制是Z-Buffer的一个主要劣势.
- Stencil-Buffer是一个用来记录被绘制的图元的位置的Offscreen-Buffer.它一般每个像素包含8位.图元可以使用多种方式绘制进模板缓冲,然后用来控制绘制Color-Buffer和Z-Buffer.例如,假定一个填了色的圆被绘制进了Stencil-Buffer,可以用来控制只将接下来的图元绘制到圆限制的区域.

## Chapter5 Shading Basics

### Screen-Based Antialiasing
- Low sampling rate (The general strategy of screen-based antialiasing schemes is to use a sampling pattern for the screen and then weight and sum the samples to produce a pixel color)![MultiSample](./Images/MultiSample.png)These weights sum to one. Most methods used in real-time rendering systems give a uniform weight to their samples. The default mode for graphics hardware, a single sample at the center of the pixel, is the simplest case of the antialiasing equation.Antialiasing algorithms that compute more than one full sample per pixel are called supersampling (or oversampling) methods.
- Conceptually simplest, full-scene antialiasing (FSAA), also known as “supersampling antialiasing” (SSAA), renders the scene at a higher resolution and then filters neighboring samples to create an image. For example, say an image of 1280 × 1024 pixels is desired. If you render an image of 2560×2048 offscreen and then average each 2×2 pixel area on the screen, the desired image is generated with four samples per pixel, filtered using a box filter. This method is costly, as all subsamples must be fully shaded and filled, with a z-buffer depth per sample. FSAA’s main advantage is simplicity.
- A sampling method related to supersampling is based on the idea of the accumulation buffer. Instead of one large offscreen buffer, this method uses a buffer that has the same resolution as the desired image, but with more bits of color per channel. To obtain a 2 × 2 sampling of a scene, four images are generated, with the view moved half a pixel in the screen x- or y-direction as needed.
- Multisampling antialiasing (MSAA) lessens the high computational costs by computing the surface’s shade once per pixel and sharing this result among the samples. Pixels may have, say, four (x,y) sample locations per fragment, each with their own color and z-depth, but the pixel shader is evaluated only once for each object fragment applied to the pixel.
- Once all geometry has been rendered to a multiple-sample buffer, a resolve operation is then performed. This procedure averages the sample colors together to determine the color for the pixel. It is worth noting that a problem can arise when using multisampling with high dynamic range color values. In such cases, to avoid artifacts you normally need to tone-map the values before the resolve. This can be expensive, so a simpler approximation to the tone map function or other methods can be used.
- temporal antialiasing (TAA), MFAA (multi-frame antialiasing)
- fast approximate antialiasing (FXAA)
- subpixel morphological antialiasing (SMAA)

### Blending Order
- Blending this fragment’s value with the original pixel color is usually done using the over operator, as follows: $$c_{o}=\alpha_{s}c_{s}+(1-\alpha)c_{d}$$
where $c_{s}$ is the color of the transparent object (called the source), $\alpha_{s}$ is the object’s alpha, $c_{d}$ is the pixel color before blending (called the destination), and $c_{o}$ is the resulting color due to placing the transparent object over the existing scene.
- Another operation that sees some use is additive blending, where pixel values are simply summed. That is: $$c_{o}=\alpha_{s}c_{s}+c_{d}$$ This blending mode can work well for glowing effects such as lightning or sparks that do not attenuate the pixels behind but instead only brighten them. For several layered semitransparent surfaces, such as smoke or fire, additive blending has the effect of saturating the colors of the phenomenon.
- A limitation of the z-buffer is that only one object is stored per pixel. If several transparent objects overlap the same pixel, the z-buffer alone cannot hold and later resolve the effect of all the visible objects. When using over the transparent surfaces at any given pixel generally need to be rendered in back-to-front order. Not doing so can give incorrect perceptual cues.  One way to achieve this ordering is to sort individual objects by, say, the distance of their centroids along the view direction. This rough sorting can work reasonably well, but has a number of problems under various circumstances. First, the order is just an approximation, so objects classified as more
distant may be in front of objects considered nearer. Objects that interpenetrate are impossible to resolve on a per-mesh basis for all view angles, short of breaking each mesh into separate pieces. Nonetheless, because of its simplicity and speed, as well as needing no additional memory or special GPU support, performing a rough sort for transparency is still commonly used. If implemented, it is usually best to turn off z-depth replacement when performing transparency. That is, the z-buffer is still tested normally, but surviving surfaces do not change the z-depth stored; the closest opaque surface’s depth is left intact. In this way, all transparent objects will at least appear in some form, versus suddenly appearing or disappearing when a camera rotation changes the sort order. Other techniques can also help improve the appearance, such as drawing each transparent mesh twice as you go, first rendering backfaces and then frontfaces. The over equation can also be modified so that blending front to back gives the same result. This blending mode is called the under operator: $$c_{o}=\alpha_{d}c_{d}+(1-\alpha_{d})\alpha_{s}c_{s}$$ $$a_{o}=\alpha_{s}(1-\alpha_{d})+\alpha_{d}=\alpha_{s}-\alpha_{s}\alpha_{d}+\alpha_{d}$$
- Another use of the under operator is for performing an order-independent transparency (OIT) algorithm known as depth peeling. The idea behind depth peeling is to use two z-buffers and multiple passes. First, a rendering pass is made so that all surfaces’ z-depths, including transparent surfaces, are in the first z-buffer. In the second pass all transparent objects are rendered. If the z-depth of an object matches the value in the first z-buffer, we know this is the closest transparent object and save its RGBα to a separate color buffer. We also “peel” this layer away by saving the z-depth of whichever transparent object, if any, is beyond the first z-depth and is closest. This z-depth is the distance of the second-closest transparent object. Successive passes continue to peel and add transparent layers using under. We stop after some number of passes and then blend the transparent image atop the opaque image.

### Display Encoding
- When we calculate the effect of lighting, texturing, or other operations, the values used are assumed to be linear. Informally, this means that addition and multiplication work as expected. However, to avoid a variety of visual artifacts, display buffers and textures use nonlinear encodings that we must take into account. The short and sloppy answer is as follows: Take shader output colors in the range $[0,1]$ and raise them by a power of $1/2.2$, performing what is called gamma correction.
- When encoding linear color values for display, our goal is to cancel out the effect of the display transfer function, so that whatever value we compute will emit a corresponding radiance level. For example, if our computed value is doubled, we want the output radiance to be doubled. To maintain this connection, we apply the inverse of the display transfer function to cancel out its nonlinear effect. This process of nullifying the display’s response curve is also called gamma correction.
- It is important to apply the conversion at the final stage of rendering, when the values are written to the framebuffer for the display. If post-processing is applied after display encoding, such effects will be computed on nonlinear values, which is usually incorrect and will often cause artifacts.
![GammaCorrection](./Images/GammaCorrection.png)
![Gamma Correction Curves](./Images/gamma_correction_gamma_curves.png)
[LearnOpenGL-GammaCorrection](https://learnopengl.com/Advanced-Lighting/Gamma-Correction)

## Chapter6 Texturing

- In DirectX the upper left corner of the texture is $(0,0)$ and the lower right is $(1,1)$. This matches how many image types store their data, the top row being the first one in the file. In OpenGL the texel $(0,0)$ is located in the lower left, a y-axis flip from DirectX. 
- This brings up the question of what the floating point coordinates of the center of a pixel are. DirectX 9 defined each center at $(0.0,0.0)$—this uses rounding. This system was somewhat confusing, as the upper left corner of the upper left pixel, at DirectX’s origin, then had the value $(−0.5,−0.5)$. DirectX 10 onward changes to OpenGL’s system, where the center of a texel has the fractional values $(0.5,0.5)$—truncation, or more accurately, flooring, where the fraction is dropped. Flooring is a more natural system that maps well to language, in that pixel $(5,9)$, for example, defines a range from 5.0 to 6.0 for the u-coordinate and 9.0 to 10.0 for the v.
- A common solution to the blurriness that accompanies magnification is to use detail textures. These are textures that represent fine surface details, from scratches on a cellphone to bushes on terrain. Such detail is overlaid onto the magnified texture as a separate texture, at a different scale. The high-frequency repetitive pattern of
the detail texture, combined with the low-frequency magnified texture, has a visual effect similar to the use of a single high-resolution texture.
- Bilinear interpolation interpolates linearly in two directions. 
- The texture (at level zero) is downsampled to a quarter of the original area, with each new texel value often computed as the average of four neighbor texels in the original texture. ![Mipmaps](./Images/Mipmap.png)
- For textures encoded in a nonlinear space (such as most color textures), ignoring gamma correction when filtering will modify the perceived brightness of the mipmap levels. As you get farther away from the object and the uncorrected mipmaps get used, the object can look darker overall, and contrast and details can also be affected. For this reason, it is important to convert such textures from sRGB to linear space, perform all mipmap filtering in that space, and convert the final results back into sRGB color space for storage.
- However, mipmapping has several flaws. A major one is overblurring. Imagine a pixel cell that covers a large number of texels in the u-direction and only a few in the v-direction. This case commonly occurs when a viewer looks along a textured surface nearly edge-on. In fact, it is possible to need minification along one axis of the texture and magnification along the other. The effect of accessing the mipmap is that square areas on the texture are retrieved; retrieving rectangular areas is not possible. To avoid aliasing, we choose the largest measure of the approximate coverage of the pixel cell on the texture. This results in the retrieved sample often being relatively blurry.
- Although volume textures have significantly higher storage requirements and are more expensive to filter, they do have some unique advantages. The complex process of finding a good two-dimensional parameterization for the three-dimensional mesh can be skipped, since three-dimensional locations can be used directly as texture coordinates. This avoids the distortion and seam problems that commonly occur with two-dimensional parameterizations.
- A cube map is accessed with a three-component texture coordinate vector that specifies the direction of a ray pointing from the center of the cube outward. The texture coordinate with the largest magnitude selects the corresponding face (e.g., the vector $(−3.2,5.1,−8.4)$ selects the −z face). The remaining two coordinates are divided by the absolute value of the largest magnitude coordinate, i.e., $8.4$. They now range from $−1$ to $1$, and are simply remapped to $[0,1]$ in order to compute the texture coordinates. For example, the coordinates $(−3.2,5.1)$ are mapped to $((−3.2/8.4+1)/2, (5.1/8.4+1)/2) ≈ (0.31,0.80)$
-  Bindless textures avoid any type of binding cost in the driver, which makes rendering faster.
- One solution that directly attacks memory and bandwidth problems and caching concerns is fixed-rate texture compression. By having the GPU decode compressed textures on the fly, a texture can require less texture memory and so increase the effective cache size. At least as significant, such textures are more efficient to use, as they consume less memory bandwidth when accessed. The main drawback of these compression schemes is that they are lossy.
![TextureCompression](./Images/TextureCompression.png)
![Decal](./Images/Decal.png)
- Alpha to coverage, and the similar feature transparency adaptive antialiasing, take the transparency value of the fragment and convert this into how many samples inside a pixel are covered [1250]. This idea is like screen-door transparency, but at a subpixel level. Imagine that each pixel has four sample locations, and that a fragment covers a pixel, but is 25% transparent (75% opaque), due to the cutout texture. The alpha to coverage mode makes the fragment become fully opaque but has it cover only three of the four samples. This mode is useful for cutout textures for overlapping grassy fronds, for example. Since each sample drawn is fully opaque, the closest frond will hide objects behind it in a consistent way along its edges. No sorting is needed to correctly blend semitransparent edge pixels, since alpha blending is turned off. Alpha to coverage is good for antialiasing alpha testing, but can show artifacts when alpha blending. For example, two alpha-blended fragments with the same alpha coverage percentage will use the same subpixel pattern, meaning that one fragment will entirely cover the other instead of blending with it.

### Parallax Mapping
![Parallax Mapping](./Images/ParallaxMapping.png)
- Parallax refers to the idea that the positions of objects move relative to one another as the observer moves. As the viewer moves, the bumps should appear to have heights. The key idea of parallax mapping is to take an educated guess of what should be seen in a pixel by examining the height of what was found to be visible.
- For parallax mapping, the bumps are stored in a heightfield texture. When viewing the surface at a given pixel, the heightfield value is retrieved at that location and used to shift the texture coordinates to retrieve a different part of the surface. The amount to shift is based on the height retrieved and the angle of the eye to the surface.
- The heightfield values are either stored in a separate texture, or packed in an unused color or alpha channel of some other texture (care must be taken when packing unrelated textures together, since this can negatively impact compression quality). 
- The heightfield values are scaled and biased before being used to shift the coordinates. The scale determines how high the heightfield is meant to extend above or below the surface, and the bias gives the “sea-level” height at which no shift takes place. 
- Given a texture-coordinate location $p$, an adjusted heightfield height $h$, and a normalized view vector $v$ with a height value $v_{z}$ and horizontal component $v_{xy}$ , the new parallax-adjusted texture coordinate $p_{adj}$ is $$p_{adj}=p+\frac{h\cdot v_{xy}}{v_{z}}$$ Note that unlike most shading equations, here the space in which the computation is performed matters—the view vector needs to be in tangent space.
-  However, this method falls apart at shallow viewing angles. When the view vector is near the surface’s horizon, a small height change results in a large texture coordinate shift. The approximation fails, as the new location retrieved has little or no height correlation to the original surface location.
![Parallax Mapping](./Images/ParallaxMappingOffsetLimits.png)
To ameliorate this problem, Welsh [1866] introduced the idea of offset limiting. The idea is to limit the amount of shifting to never be larger than the retrieved height. The equation is then $$p_{adj}=p+h\cdot v_{xy}$$ Note that this equation is faster to compute than the original. Geometrically, the interpretation is that the height defines a radius beyond which the position cannot shift.

### Parallax Occlusion Mapping (Relief Mapping)
![Parallax Occlusion Mapping](./Images/ParallaxOcclusionMapping.png)
- The key idea is to first test a fixed number of heightfield texture samples along the projected vector. More samples are usually generated for view rays at grazing angles, so that the closest intersection point is not missed.
- Each three-dimensional location along the ray is retrieved, transformed into texture space, and processed to determine if it is above or below the heightfield. Once a sample below the heightfield is found, the amount it is below, and the amount the previous sample is above, are used to find an intersection location.

## Chapter7 Shadows
- Punctual light sources, i.e., those with no area, generate only fully shadowed regions, sometimes called hard shadows. If area or volume light sources are used, then soft shadows are produced.
- Each shadow can then have a fully shadowed region, called the umbra, and a partially shadowed region, called the penumbra. Soft shadows are recognized by their fuzzy shadow edges. However, it is important to note that they generally cannot be rendered correctly by just blurring the edges of a hard shadow with a low-pass filter.
- The umbra region of a soft shadow is not equivalent to a hard shadow generated by a punctual light source. Instead, the umbra region of a soft shadow decreases in size as the light source grows larger, and it might even disappear, given a large enough light source and a receiver far enough from the occluder. Soft shadows are generally preferable because the penumbrae edges let the viewer know that the shadow is indeed a shadow. Hard-edged shadows usually look less realistic and can sometimes be misinterpreted as actual geometric features, such as a crease in a surface. However, hard shadows are faster to render than soft shadows.

### Planar Shadows
- The three-dimensional object is rendered a second time to create a shadow. A matrix can be derived that projects the vertices of an object onto a plane.
- Where the light source is located at $l$, the vertex to be projected is at $v$, and the projected vertex is at $p$. We will derive the projection matrix for the special case where the shadowed plane is $y = 0$,  From the similar triangles, we get $$\frac{p_{x}-l_{x}}{v_{x}-l_{x}}=\frac{l_{y}}{l_{y}-v{y}}\Longleftrightarrow p_{x}=\frac{l_{y}v_{x}-l_{x}v_{y}}{l_{y}-v_{y}}$$ Now these equations can be converted into the projection matrix $M$:
$$M=
\begin{vmatrix}
   l_{y} & -l_{x} & 0 & 0 \newline
   0 & 0 & 0 & 0 \newline
   0 & -l_{z} & l_{y} & 0 \newline
   p & -l & 0 & l_{y}
  \end{vmatrix}
$$
- In the general case, the plane onto which the shadows should be cast is not the plane $y = 0$, but instead $\pi : n · x + d = 0$. The goal is again to find a matrix that projects $v$ down to $p$. To this end, the ray emanating at $l$, which goes through $v$, is intersected by the plane $\pi$. This yields the projected point $p$: $$p=l-\frac{d+n\cdot l}{n\cdot (v-l)}(v-l)$$
This equation can also be converted into a projection matrix, which satisfies $Mv = p$:
$$M=
\begin{vmatrix}
   n\cdot l+d-l_{x}n_{x} & -l_{x}n_{y} & -l_{x}n_{z} & -l_{x}d \newline
   -l_{y}n_{x} & n\cdot l+d-l_{y}n_{y} & -l_{y}n_{z} & -l_{y}d \newline
   -l_{z}n_{x} & -l_{z}n_{y} & n\cdot l+d-l_{z}n_{z} & -l_{z}d \newline
   -n_{x} & -n_{y} & -n_{z} & n\cdot l
  \end{vmatrix}
$$
![Shadow Projection](./Images/ShadowProjection.png)
- To render the shadow, simply apply this matrix to the objects that should cast shadows on the plane π, and render this projected object with a dark color and no illumination. In practice, you have to take measures to avoid allowing the projected triangles to be rendered beneath the surface receiving them. One method is to add some bias to the plane we project upon, so that the shadow triangles are always rendered in front of the surface. A safer method is to draw the ground plane first, then draw the projected triangles with the z-buffer off, then render the rest of the geometry as usual. The projected triangles are then always drawn on top of the ground plane, as no depth comparisons are made.
- If the ground plane has a limit, e.g., it is a rectangle, the projected shadows may fall outside of it, breaking the illusion. To solve this problem, we can use a stencil buffer. First, draw the receiver to the screen and to the stencil buffer. Then, with the z-buffer off, draw the projected triangles only where the receiver was drawn, then render the rest of the scene normally.
- Another shadow algorithm is to render the triangles into a texture, which is then applied to the ground plane. This texture is a type of light map, a texture that modulates the intensity of the underlying surface. As will be seen, this idea of rendering the shadow projection to a texture also allows penumbrae and shadows on curved surfaces. One drawback of this technique is that the texture can become magnified, with a single texel covering multiple pixels, breaking the illusion. If the shadow situation does not change from frame to frame, i.e., the light and shadow casters do not move relative to each other, this texture can be reused. Most shadow techniques can benefit from reusing intermediate computed results from frame to frame if no change has occurred.

### Soft Shadows
- Soft shadows appear whenever a light source has an area. One way to approximate the effect of an area light is to sample it by using several punctual lights placed on its surface. For each of these punctual light sources, an image is rendered and accumulated into a buffer. The average of these images is then an image with soft shadows. In practice, doing so at interactive rates is usually untenable because of the execution time that would be involved.
- Heckbert and Herf use a frustum-based method to produce their shadows. The idea is to treat the light as the viewer, and the ground plane forms the far clipping plane of the frustum. The frustum is made wide enough to encompass the occluders. A soft shadow texture is formed by generating a series of ground-plane textures. The area light source is sampled over its surface, with each location used to shade the image representing the ground plane, then to project the shadow-casting objects onto this image. All these images are summed and averaged to produce a ground-plane shadow texture.
- A more efficient approach is to use convolution, i.e., filtering. Blurring a hard shadow generated from a single point can be sufficient in some cases and can produce a semitransparent texture that can be composited with real-world content.
- One simple way to extend the idea of planar shadows to curved surfaces is to use a generated shadow image as a projective texture

### Shadow Volume
![Shadow Volume](./Images/ShadowVolume.png)
- Say we view some scene and follow a ray from the eye through a pixel until the ray hits the object to be displayed on screen. While the ray is on its way to this object, we increment a counter each time it crosses a face of the shadow volume that is frontfacing (i.e., facing toward the viewer). Thus, the counter is incremented each time the ray goes into shadow. In the same manner, we decrement the same counter each time the ray crosses a backfacing face of the truncated pyramid. The ray is then going out of a shadow. We proceed, incrementing and decrementing the counter until the ray hits the object that is to be displayed at that pixel. If the counter is greater than zero, then that pixel is in shadow; otherwise it is not. This principle also works when there is more than one triangle that casts shadows.
- Doing this with rays is time consuming. But there is a much smarter solution: A stencil buffer can do the counting for us. 
	- First, the stencil buffer is cleared. 
	- Second, the whole scene is drawn into the framebuffer with only the color of the unlit material used, to get these shading components in the color buffer and the depth information into the z-buffer. 
	- Third, z-buffer updates and writing to the color buffer are turned off (though z-buffer testing is still done), and then the frontfacing triangles of the shadow volumes are drawn. During this process, the stencil operation is set to increment the values in the stencil buffer wherever a triangle is drawn. 
	- Fourth, another pass is done with the stencil buffer, this time drawing only the backfacing triangles of the shadow volumes. For this pass, the values in the stencil buffer are decremented when the triangles are drawn. Incrementing and decrementing are done only when the pixels of the rendered shadow-volume face are visible (i.e., not hidden by any real geometry). At this point the stencil buffer holds the state of shadowing for every pixel. 
	- Finally, the whole scene is rendered again, this time with only the components of the active materials that are affected by the light, and displayed only where the value in the stencil buffer is 0. A value of 0 indicates that the ray has gone out of shadow as many times as it has gone into a shadow volume—i.e., this location is illuminated by the light.

### Shadow Maps
- In 1978, Williams proposed that a common z-buffer-based renderer could be used to generate shadows quickly on arbitrary objects. The idea is to render the scene, using the z-buffer, from the position of the light source that is to cast shadows. Whatever the light “sees” is illuminated, the rest is in shadow. When this image is generated, only z-buffering is required. Lighting, texturing, and writing values into the color buffer can be turned off.
- Not all objects in the scene need to be rendered into the light’s view volume. First, only objects that can cast shadows need to be rendered. For example, if it is known that the ground can only receive shadows and not cast one, then it does not have to be rendered into the shadow map.
- One disadvantage of shadow mapping is that the quality of the shadows depends on the resolution (in pixels) of the shadow map and on the numerical precision of the z-buffer. 
- Since the shadow map is sampled during the depth comparison, the algorithm is susceptible to aliasing problems, especially close to points of contact between objects. 
- A common problem is self-shadow aliasing, often called “surface acne” or “shadow acne,” in which a triangle is incorrectly considered to shadow itself. This problem has two sources. One is simply the numerical limits of precision of the processor. The other source is geometric, from the fact that the value of a point sample is being used to represent an area’s depth. That is, samples generated for the light are almost never at the same locations as the screen samples (e.g., pixels are often sampled at their centers). When the light’s stored depth value is compared to the viewed surface’s depth, the light’s value may be slightly lower than the surface’s, resulting in self-shadowing. 
	- One common method to help avoid (but not always eliminate) various shadow-map artifacts is to introduce a bias factor. When checking the distance found in the shadow map with the distance of the location being tested, a small bias is subtracted from the receiver’s distance. This bias could be a constant value, but doing so can fail when the receiver is not mostly facing the light. A more effective method is to use a bias that is proportional to the angle of the receiver to the light. The more the surface tilts away from the light, the greater the bias grows, to avoid the problem. This type of bias is called the slope scale bias.

## Chapter8 Light and Color
- Irradiance is the density of radiant flux with respect to area ${d\phi}/{dA}$.
- 
| Name | Symbol | Units |
|------|--------|-------|
| radiant flux | $\phi$| watt($W$) |
| irradiance | $E$ | $W/m^2$ |
| radiant intensity | $I$ | $W/m^2$ |
| radiance | $L$ | $W/(m^2sr)$ |
- An important property of radiance is that it is not affected by distance, ignoring atmospheric effects such as fog. In other words, a surface will have the same radiance regardless of its distance from the viewer. The surface covers fewer pixels when more distant, but the radiance from the surface at each pixel is constant.

